In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
data = pd.read_csv("Ecommerce_Delivery_Analytics_New.csv")
data.head()

,Order ID,Customer ID,Platform,Order Date & Time,Delivery Time (Minutes),Product Category,Order Value (INR),Customer Feedback,Service Rating,Delivery Delay,Refund Requested
0,ORD000001,CUST2824,JioMart,19:29.5,30,Fruits & Vegetables,382,"Fast delivery, great service!",5,No,No
1,ORD000002,CUST1409,Blinkit,54:29.5,16,Dairy,279,Quick and reliable!,5,No,No
2,ORD000003,CUST5506,JioMart,21:29.5,25,Beverages,599,Items missing from order.,2,No,Yes
3,ORD000004,CUST5012,JioMart,19:29.5,42,Beverages,946,Items missing from order.,2,Yes,Yes
4,ORD000005,CUST4657,Blinkit,49:29.5,30,Beverages,334,"Fast delivery, great service!",5,No,No


In [8]:
for col in data.select_dtypes(include="object").columns:
    data[col] = data[col].astype(str).str.strip()
    data[col] = data[col].replace({"": pd.NA, "nan": pd.NA, "none": pd.NA, "n/a": pd.NA, "na": pd.NA})

data.select_dtypes(include="object").nunique().sort_values(ascending=False).head(20)

date_cols = [c for c in data.columns if any(x in c for x in ("date","time","timestamp","datetime"))]
for c in date_cols:
    data[c] = pd.to_datetime(data[c], errors="coerce")
    print(c, data[c].isna().mean())

def to_numeric_clean(s):
    return pd.to_numeric(s.astype(str).str.replace(",","").str.replace("₹","").str.replace("$","").str.replace("%",""), errors="coerce")

possible_num = ["Order Value (INR)", "Delivery Time (Minutes)", "Service Rating"]  # adapt to your columns
for c in possible_num:
    data[c] = to_numeric_clean(data[c])
    print(c, data[c].dtype, data[c].isna().sum())


Order Value (INR) int64 0
Delivery Time (Minutes) int64 0
Service Rating int64 0


In [9]:
dup_count = data.duplicated().sum()
df = data.drop_duplicates().reset_index(drop=True)
print("duplicates removed:", dup_count)

missing = df.isna().sum().sort_values(ascending=False)
print(missing.head(30))


duplicates removed: 0
Order ID                   0
Customer ID                0
Platform                   0
Order Date & Time          0
Delivery Time (Minutes)    0
Product Category           0
Order Value (INR)          0
Customer Feedback          0
Service Rating             0
Delivery Delay             0
Refund Requested           0
dtype: int64


In [23]:
# Example: delivery_delay -> map variants
df["Delivery Delay"] = df["Delivery Delay"].str.lower().map({
    "yes":"Yes", "y":"Yes", "no":"No", "n":"No", "true":"Yes", "false":"No"
})
# For product_category: strip, title case and collapse typos
df["Product Category"] = df["Product Category"].str.strip().str.title()
# If many similar labels, group small categories into "Other"
vc = df["Product Category"].value_counts(normalize=True)
small = vc[vc < 0.01].index
df["Product Category"] = df["Product Category"].replace(list(small), "Other")

def cap_iqr(series, k=3):
    q1,q3 = series.quantile([0.25,0.75])
    iqr = q3-q1
    lower, upper = q1 - k*iqr, q3 + k*iqr
    return series.clip(lower, upper)

df["Delivery Time (Minutes)"] = cap_iqr(df["Delivery Time (Minutes)"], k=3)

# Clean invalid time values in "Order Date & Time"
df["Order Date & Time"] = df["Order Date & Time"].apply(
    lambda x: x if pd.to_datetime(x, errors="coerce") is not pd.NaT else None
)

# Drop rows with invalid "Order Date & Time"
df = df.dropna(subset=["Order Date & Time"])

# Parse "Order Date & Time" and extract date and hour
df["Order Date"] = pd.to_datetime(df["Order Date & Time"], errors="coerce").dt.date
df["Order Hour"] = pd.to_datetime(df["Order Date & Time"], errors="coerce").dt.hour

# buckets
df["Delivery Speed Bucket"] = pd.cut(df["Delivery Time (Minutes)"],
                                     bins=[-1,30,60,99999],
                                     labels=["<30","30-60",">60"])
df["is_refund"] = df["Refund Requested"].str.lower().map({"yes":1,"no":0})
df["delay_flag"] = df["Delivery Delay"].str.lower().map({"yes":1,"no":0})




C:\Users\viren\AppData\Local\Temp\ipykernel_20580\182501463.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Order Date"] = pd.to_datetime(df["Order Date & Time"], errors="coerce").dt.date
C:\Users\viren\AppData\Local\Temp\ipykernel_20580\182501463.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Order Hour"] = pd.to_datetime(df["Order Date & Time"], errors="coerce").dt.hour


In [27]:
df.to_csv("All_Ecommerce_NEW_cleaned_by_you.csv", index=False)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39911 entries, 0 to 99998
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Order ID                 39911 non-null  object  
 1   Customer ID              39911 non-null  object  
 2   Platform                 39911 non-null  object  
 3   Order Date & Time        39911 non-null  object  
 4   Delivery Time (Minutes)  39911 non-null  int64   
 5   Product Category         39911 non-null  object  
 6   Order Value (INR)        39911 non-null  int64   
 7   Customer Feedback        39911 non-null  object  
 8   Service Rating           39911 non-null  int64   
 9   Delivery Delay           39911 non-null  object  
 10  Refund Requested         39911 non-null  object  
 11  Order Date               39911 non-null  object  
 12  Order Hour               39911 non-null  int32   
 13  Delivery Speed Bucket    39911 non-null  category
 14  is_refund  